<a href="https://colab.research.google.com/github/sergejhorvat/TensorFlow-in-Practice---Coursera/blob/master/Convolutional%20Neural%20Networks%20in%20TensorFlow/Exercise_8_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
tf.__version__

     |████████████████████████████████| 380.8MB 46kB/s 
     |████████████████████████████████| 450kB 42.3MB/s 
     |████████████████████████████████| 3.8MB 47.4MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goog

'2.0.0'

The data for this exercise is available at: https://www.kaggle.com/datamunge/sign-language-mnist/

Sign up and download to find 2 CSV files: sign_mnist_test.csv and sign_mnist_train.csv -- You will upload both of them using this button before you can continue.


In [2]:
#uploaded=files.upload()

!wget --no-check-certificate \
    http://mojemjesto.net/Coursera/TensorFlow_in_practice/sign_mnist_train.csv.zip \
    -O /tmp/sign_mnist_train.csv.zip
  
!wget --no-check-certificate \
    http://mojemjesto.net/Coursera/TensorFlow_in_practice/sign_mnist_test.csv.zip \
    -O /tmp/sign_mnist_test.csv.zip



--2019-11-19 15:50:11--  http://mojemjesto.net/Coursera/TensorFlow_in_practice/sign_mnist_train.csv.zip
Resolving mojemjesto.net (mojemjesto.net)... 69.163.152.21
Connecting to mojemjesto.net (mojemjesto.net)|69.163.152.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25759917 (25M) [application/zip]
Saving to: ‘/tmp/sign_mnist_train.csv.zip’

/tmp/sign_mnist_tra 100%[===================>]  24.57M  11.1MB/s    in 2.2s    

2019-11-19 15:50:13 (11.1 MB/s) - ‘/tmp/sign_mnist_train.csv.zip’ saved [25759917/25759917]

--2019-11-19 15:50:15--  http://mojemjesto.net/Coursera/TensorFlow_in_practice/sign_mnist_test.csv.zip
Resolving mojemjesto.net (mojemjesto.net)... 69.163.152.21
Connecting to mojemjesto.net (mojemjesto.net)|69.163.152.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6688121 (6.4M) [application/zip]
Saving to: ‘/tmp/sign_mnist_test.csv.zip’

/tmp/sign_mnist_tes 100%[===================>]   6.38M  4.93MB/s    in 1.3s    

2

In [0]:
import os
import zipfile

local_zip = '/tmp/sign_mnist_train.csv.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

local_zip = '/tmp/sign_mnist_test.csv.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

In [12]:
os.chdir('/tmp/')
print(os.getcwd())
from numpy import genfromtxt

/tmp


In [14]:
from numpy import genfromtxt
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  #
  # The first value is the label
  #
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion

  # I will use numpy to get data no bulky read line or other methods 
  data = genfromtxt(filename, delimiter=',')
  labels = data[1:,0]
  images = data[1:,1:785]

  return images, labels


training_images, training_labels = get_data('sign_mnist_train.csv')
testing_images, testing_labels = get_data('sign_mnist_test.csv')

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

(27456, 784)
(27456,)
(7173, 784)
(7173,)


In [0]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims

training_images = # Your Code Here
testing_images = # Your Code Here

# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
    # Your Code Here
    )

validation_datagen = ImageDataGenerator(
    # Your Code Here)
    
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

In [0]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([
    # Your Code Here
    )

# Compile Model. 
model.compile(# Your Code Here)

# Train the Model
history = model.fit_generator(# Your Code Here)

model.evaluate(testing_images, testing_labels)
    
# The output from model.evaluate should be close to:
[6.92426086682151, 0.56609035]


In [0]:
# Plot the chart for accuracy and loss on both training and validation

import matplotlib.pyplot as plt
acc = # Your Code Here
val_acc = # Your Code Here
loss = # Your Code Here
val_loss = # Your Code Here

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()